In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from torch.utils.tensorboard import SummaryWriter  # For logging, adjust according to k-fold

# Load dataset
# df = pd.read_csv('AmazonDataSales_v2.csv', low_memory=False)  # Assuming CSV is in the correct path
# Simulating dataset load with placeholder values
df = pd.DataFrame({
    'amount': np.random.rand(1000),
    'category': np.random.choice(['A', 'B', 'C'], 1000),
    'size': np.random.choice(['Small', 'Medium', 'Large'], 1000),
    'qty': np.random.randint(1, 10, 1000),
})

# Preprocess dataset
df_encoded = pd.get_dummies(df, columns=['category', 'size', 'qty'])
X = df_encoded.drop('amount', axis=1).values.astype(np.float32)
y = df['amount'].values.astype(np.float32)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y).unsqueeze(1)  # Ensure target tensor is correctly shaped

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model
class FeedForwardRegressor(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super(FeedForwardRegressor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, 1)  # Output layer for regression

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.fc3(x)

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

# Evaluation function
from sklearn.metrics import r2_score

def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    targets_list = []
    outputs_list = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            targets_list.append(targets.cpu().numpy())
            outputs_list.append(outputs.cpu().numpy())
    
    # Concatenate all the outputs and targets to compute overall metrics
    all_outputs = np.concatenate(outputs_list, axis=0)
    all_targets = np.concatenate(targets_list, axis=0)
    
    avg_loss = total_loss / len(test_loader)
    r2 = r2_score(all_targets, all_outputs)
    
    print(f'Average Test Loss: {avg_loss}')
    print(f'R2 Score: {r2}')
    return avg_loss, r2


# K-Fold Cross-Validation
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(kfold.split(X_tensor)):
    print(f"Fold {fold + 1}/{num_folds}")
    
    # Splitting the data for this fold
    X_train, X_test = X_tensor[train_ids], X_tensor[test_ids]
    y_train, y_test = y_tensor[train_ids], y_tensor[test_ids]
    
    # DataLoader
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64, shuffle=False)
    
    # Model, Loss, Optimizer
    input_size = X_train.shape[1]
    model = FeedForwardRegressor(input_size, hidden_size1=2, hidden_size2=2).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train and evaluate
    train_model(model, train_loader, criterion, optimizer, num_epochs=300)
    avg_loss = evaluate_model(model, test_loader, criterion)
    fold_results.append(avg_loss)

# Reporting
print(f"K-Fold Cross-Validation results: {fold_results}")
print(f"Average Loss across folds: {np.mean(fold_results)}")


Fold 1/5
Epoch 1/300, Loss: 0.6998688257657565
Epoch 2/300, Loss: 0.679802289375892
Epoch 3/300, Loss: 0.6598549347657424
Epoch 4/300, Loss: 0.6377635873281039
Epoch 5/300, Loss: 0.6254522296098562
Epoch 6/300, Loss: 0.6050324898499709
Epoch 7/300, Loss: 0.5860171959950373
Epoch 8/300, Loss: 0.5648997013385479
Epoch 9/300, Loss: 0.5477574834456811
Epoch 10/300, Loss: 0.5317598741788131
Epoch 11/300, Loss: 0.5241143336662879
Epoch 12/300, Loss: 0.5016779670348535
Epoch 13/300, Loss: 0.48570096492767334
Epoch 14/300, Loss: 0.47090478585316586
Epoch 15/300, Loss: 0.462982890697626
Epoch 16/300, Loss: 0.4424450695514679
Epoch 17/300, Loss: 0.43076157799133885
Epoch 18/300, Loss: 0.41286881611897397
Epoch 19/300, Loss: 0.4031100915028499
Epoch 20/300, Loss: 0.3972616264453301
Epoch 21/300, Loss: 0.3799059666120089
Epoch 22/300, Loss: 0.37135080190805286
Epoch 23/300, Loss: 0.3582452741953043
Epoch 24/300, Loss: 0.3436262641961758
Epoch 25/300, Loss: 0.33545022973647487
Epoch 26/300, Loss: 0